In [26]:
#Install package
#! pip install tokenizers
#!pip install torch
! pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 7.0 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 60.1 MB/s eta 0:00:01


In [2]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./eo_data/").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model(".", "esperberto")

['./esperberto-vocab.json', './esperberto-merges.txt']

In [20]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./esperberto-vocab.json",
    "./esperberto-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

print(
    tokenizer.encode("Mi estas Julien.")
)
# Encoding(num_tokens=7, ...)
# tokens: ['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [30]:
import torch
from torch.utils.data import Dataset

class EsperantoDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "./esperberto-vocab.json",
            "./esperberto-merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []

        src_files = Path("./data/").glob("*-eval.txt") if evaluate else Path("./data/").glob("*-train.txt")
        for src_file in src_files:
            print("🔥", src_file)
            lines = src_file.read_text(encoding="utf-8").splitlines()
            self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

In [45]:
fill_mask("Jen la komenco de bela <mask>.")


[{'score': 0.07220534980297089,
  'token': 14379,
  'token_str': 'ña',
  'sequence': 'Jen la komenco de belaña.'},
 {'score': 0.04695172980427742,
  'token': 14182,
  'token_str': 'ño',
  'sequence': 'Jen la komenco de belaño.'},
 {'score': 0.03377094119787216,
  'token': 19393,
  'token_str': 'ça',
  'sequence': 'Jen la komenco de belaça.'},
 {'score': 0.020697252824902534,
  'token': 17819,
  'token_str': ' persona',
  'sequence': 'Jen la komenco de bela persona.'},
 {'score': 0.018034767359495163,
  'token': 42364,
  'token_str': ' bene',
  'sequence': 'Jen la komenco de bela bene.'}]

In [47]:
result = fill_mask("La suno <mask>.")
print(result)

[{'score': 0.06674236059188843, 'token': 14379, 'token_str': 'ña', 'sequence': 'La sunoña.'}, {'score': 0.023482680320739746, 'token': 2478, 'token_str': 'za', 'sequence': 'La sunoza.'}, {'score': 0.016468700021505356, 'token': 25610, 'token_str': ' yo', 'sequence': 'La suno yo.'}, {'score': 0.014822062104940414, 'token': 23819, 'token_str': ' dice', 'sequence': 'La suno dice.'}, {'score': 0.013930008746683598, 'token': 2727, 'token_str': 'ón', 'sequence': 'La sunoón.'}]
